In [0]:
%pip install -qqqq -U databricks-sdk unitycatalog-ai[databricks]
dbutils.library.restartPython()

In [0]:
%load_ext autoreload
%autoreload 2
import warnings

warnings.simplefilter(action="ignore")

In [0]:
dbutils.widgets.text("space_id", "", "space_id")
dbutils.widgets.text(name="catalog", defaultValue="bo_cheng_dnb_demos", label="catalog")
dbutils.widgets.text(name="schema", defaultValue="agents", label="schema")

In [0]:
space_id = dbutils.widgets.get("space_id")

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
content = "Can you select trip distance and fare amount from the trips table"
initial_conversation = w.genie.start_conversation_and_wait(
    space_id=space_id, content=content
)

In [0]:
initial_attachment_query_result = w.genie.get_message_attachment_query_result(
    space_id=space_id,
    conversation_id=initial_conversation.conversation_id,
    message_id=initial_conversation.message_id,
    attachment_id=initial_conversation.attachments[0].attachment_id,
)

In [0]:
result_data_array = initial_attachment_query_result.statement_response.result.data_array

In [0]:
import pandas as pd

df = pd.DataFrame(result_data_array, columns=["x", "y"])
df["x"] = df["x"].astype(float)
df["y"] = df["y"].astype(float)

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

CATALOG = dbutils.widgets.get("catalog")
SCHEMA = dbutils.widgets.get("schema")
my_callable = client.get_function_as_callable(
    function_name=f"{CATALOG}.{SCHEMA}.plot_to_json"
)

In [0]:
import json

input_json_str = json.dumps({"rows": json.loads(df.to_json(orient="records"))})

In [0]:
import plotly.io as pio

json_string = my_callable(payload_json=input_json_str)
fig = pio.from_json(json_string)
fig.show()